# Система для рекомендации новостей

Для начала считаем все данные датасета, как мы делали в предыдущем ноутбуке.

In [42]:
import os, json
import numpy as np
import pandas as pd

In [44]:
def get_filenames(path):
    filenames = [path+pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]
    return filenames

In [45]:
path_to_json = 'data/'
json_files = get_filenames(path_to_json)
print(json_files[:10])

['data/news_0283889.json', 'data/news_0175708.json', 'data/news_0194374.json', 'data/news_0007868.json', 'data/news_0122036.json', 'data/news_0254026.json', 'data/news_0103335.json', 'data/news_0119287.json', 'data/news_0033948.json', 'data/news_0241999.json']


In [4]:
with open(json_files[0]) as f:
    file = json.load(f)
file

{'organizations': [],
 'uuid': '7de061b1f81736fc573dad0938918666b594ee7a',
 'thread': {'social': {'gplus': {'shares': 0},
   'pinterest': {'shares': 0},
   'vk': {'shares': 0},
   'linkedin': {'shares': 0},
   'facebook': {'likes': 0, 'shares': 0, 'comments': 0},
   'stumbledupon': {'shares': 0}},
  'site_full': 'finance.bigmir.net',
  'main_image': 'http://bm.img.com.ua/berlin/storage/finance/orig/8/3a/2e6a5dd12c1deb9d1ca50615a40ae3a8.jpg',
  'site_section': 'http://finance.bigmir.net/finance.bigmir.net?_ctr=rss',
  'section_title': 'Финансы',
  'url': 'http://finance.bigmir.net/news/75000-V-2017-godu-Nacbank-rasschityvaet-poluchit--chetyre-transha-ot-MVF',
  'country': 'UA',
  'domain_rank': 3864,
  'title': 'В 2017 году Нацбанк рассчитывает получить четыре транша от МВФ',
  'performance_score': 0,
  'site': 'bigmir.net',
  'participants_count': 0,
  'title_full': 'В 2017 году Нацбанк рассчитывает получить четыре транша от МВФ',
  'spam_score': 0.0,
  'site_type': 'news',
  'publishe

Однако, в этот раз нас будут интересовать только названия статей.

In [5]:
def read_dataset(filenames):
    files = []
    for file_name in filenames:
        with open(file_name) as file:
            file_dict = json.load(file)
            files.append(file_dict)
    return files


def get_features(files):
    data = [{
        'title': file['title'],
        } for file in files]
    return data


def dataset_to_df(files):
    df = pd.DataFrame(files)
    return df

In [6]:
# функция, объединяющая все вышеперечисленное
def get_files_df(filenames):
    files = read_dataset(filenames)
    dicts = get_features(files)
    df = dataset_to_df(dicts)
    return df

Чтобы уменьшить нагрузку на память и увеличить скорость работы, будем использовать лишь часть датасета. Важно заметить, что выборка данных невелика и это еще скажется на качестве обученной модели.

In [7]:
data_df = get_files_df(json_files[:10000])

In [8]:
data_df.head(10)

,title
0,В 2017 году Нацбанк рассчитывает получить четы...
1,Полиция устроила погоню со стрельбой за тракто...
2,Как прыгнуть с парашютом. Гройсман рассказал о...
3,Из олимпийского Mercedes фигуристки Столбовой ...
4,Родина взяла всего два гейма в матче с А. Радв...
5,Гостья из будущего - летающая лампочка. Смотри...
6,Вопрос по выбору страниц для продвижения (прое...
7,В Петушинском районе задержали закладчиков гер...
8,Возбуждено уголовное дело после потери зрения ...
9,"Чувашия увеличила республиканский бюджет на 1,..."


Данные получили, идем дальше.

## Токенизация (+ лемматизация)

Любые текстовые данные стоит обязательно очищать. Я буду использовать word_tokenize из библеотеки nltk, уберу все стоп-слова, лишнюю пунктуацию и пробелы которые могут возникнуть как отдельные слова.

Также, стоит учесть, что для дальнейшей обработки данных было бы неплохо провести лемматизацию текстов. Это обусловлено особенностями русского языка. Например, в английском языке эта мера была бы менее обязательной, но все же применимой.

In [9]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to /home/temur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для лемматизации будет использовать библеотеку pymystem3. Это морфологический анализатор русского языка, используемый в Yandex (возможно я не прав). Как говорится, ссылочка в описании в конце ноутбука.

In [10]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

Далее вы можете видеть функцию для препроцессинга текста. Возможно что-то не очень оптимизировано, но все же работает.

In [11]:
# функция токенизации текстов
def preprocess_text(text):
    tokens = word_tokenize(text)
    text = " ".join(tokens)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    return tokens

Пример:

In [12]:
preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")

['сказать',
 'видеть',
 'кто-то',
 'наступать',
 'грабли',
 'разочаровывать',
 'натравлять']

In [13]:
# from sklearn.model_selection import train_test_split
# # train, test = train_test_split(data_df, test_size=0.1)

Вообщем-то мы не особо нуждаемся в разделении датасета на train и test части, ибо я и так сокращал размер загружаемой выборки. А еще мы можем протестировать модель на некоторой части данных, на которых она уже будет обучена. Это не будет существенно влиять на результаты, тем более если нам их не с чем сравнивать :)

## FastText Эмбеддинг

Перейдем к самой требовательной в части объема памяти моменту. С сайта deeppavlov.ai загрузим предобученную модель на Wiki+Lenta FastText русского языка. Там было несколько моделей, я выбрал ту, у которой препроцессинг содержит "tokenize (nltk word_tokenize), lemmatize (pymorphy2)", так как я сам предобработал свои данные таким образом и это существенно уменьшает размер словаря модели.

In [14]:
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder
from deeppavlov.models.embedders.tfidf_weighted_embedder import TfidfWeightedEmbedder

[nltk_data] Downloading package punkt to /home/temur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/temur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/temur/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/temur/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


Это путь к моей загруженной модели. Она будет лежать в той же директории, что и сам ноутбук. 
Ссылка для скачивания модели: http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lemmatize/ft_native_300_ru_wiki_lenta_lemmatize.bin

In [15]:
cap_path = "ft_native_300_ru_wiki_lenta_lemmatize.bin"

In [16]:
fasttext_embedder = FasttextEmbedder(cap_path)

2019-04-19 18:08:19.155 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/home/temur/Documents/Kaggle/Internship Test - EORA/ft_native_300_ru_wiki_lenta_lemmatize.bin`]


## TF-IDF Векторизатор для TF-IDF Weighted Эмбеддинга

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
# параметры для векторизатора
params = {}
params['tokenizer'] = preprocess_text
params['stop_words'] = russian_stopwords
params['ngram_range'] = (1, 3)
params['min_df'] = 3

Далее вы можете видеть небольшой костыль, который мне пришлось создать чтобы пихнуть его в TfidfWeightedEmbedder. Я так сказать завернул мой векторизатор в оболочку с теми функциями которые необходимы для эмбеддера. На самом деле, правильней было бы унаследовать класс от deeppavlov.core.models.component.Component и инициализировать все, что нужно.

In [19]:
class Vectorizer(TfidfVectorizer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.model = self
    
    def __call__(self, sample):
        return self.transform(sample)

In [20]:
tfidf = Vectorizer(**params)

In [21]:
tfidf.fit([i for i in data_df['title']])

Vectorizer()

In [22]:
fasttext_tfidf = TfidfWeightedEmbedder(embedder=fasttext_embedder,
        vectorizer=tfidf)

In [23]:
fasttext_tfidf([['большой фильм']])[0]

array([[-0.11647623, -0.11186747,  0.07642709,  0.1264918 , -0.39655843,
         0.06397477,  0.16238   ,  0.06901422,  0.01792966,  0.10028829,
         0.16900687,  0.20173708, -0.09936717, -0.05134368, -0.16861708,
         0.10392752, -0.06480429,  0.17310715,  0.18004686, -0.07454307,
        -0.07744697,  0.1208401 , -0.11450679, -0.2510907 , -0.0626642 ,
        -0.09429549,  0.14915459,  0.11352558,  0.09210832,  0.5100619 ,
         0.2052138 ,  0.0385538 ,  0.05495359,  0.14050959, -0.09225752,
         0.07475535,  0.06417041, -0.1401072 ,  0.10847577, -0.3682249 ,
         0.21126395,  0.08137783, -0.04384461,  0.09862223, -0.02120385,
        -0.09360278, -0.03669334, -0.05648015, -0.1173844 , -0.1363184 ,
        -0.09581605,  0.19098663, -0.06546298,  0.31218368, -0.09501957,
        -0.11795574, -0.1863031 , -0.17875998, -0.06008833,  0.00881153,
         0.02532352, -0.05741346,  0.22504799, -0.35226208,  0.01570547,
        -0.2264547 ,  0.02897066,  0.11514554, -0.0

Наш FastText эмбеддер для предложений готов!

## Модель для построения пространства и нахождения соседей

Здесь можно было бы использовать KMeans классификатор и юзать его для нахождения соседей векторов. Однако, я решил, что можно добавить небольшое улучшение.

Я решил использовать алгоритм Approximate Nearest Neighbor, который, как понятно из названия, аппроксимирует результат и может не выдать точные результаты ближайших векторов. Однако! Он работает быстрей и требует меньше памяти за счет применения LSH (locality-sensitive hashing) и BBF (best bin first) алгоритмов.

На мой вгляд, данные преимущества (скорость и меньшее подтребление памяти) будут очень кстати для больших объемов данных, что весьма вероятно в данной задачи.

Я нашел удобный фрейворк NearPy, который реализует данный алгоритм, предлагая удобный функционал.

In [24]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

In [34]:
# Размерность наших векторов (взято с вывода эмбеддинга)
dimension = 300

# Создадим рандомный бинарный хеш с 2 битами
rbp = RandomBinaryProjections('rbp', 2)

# создадим engine
engine = Engine(dimension, lshashes=[rbp])

На самом деле бинарный хеш сильно влияет на конечный результат. Я заметил, что чем больше количество битов, тем хуже получаются результаты и, видимо, больше векторов попадает в одну аппроксимированную кучу. Поэтому я уменьшил количество битов, но конечно это сказывается на том, что прироста в производительности мы не получим. Однако, при большем количестве данных, мы могли бы себе позволить увеличить кол-во битов (точнее вынуждены были бы).

In [35]:
# функция добавления векторов в engine
def store_vectors(df, engine, feature='title'):
    for index, row in df.iterrows():
        tfidf_vec = fasttext_tfidf([[row[feature]]])[0]
        vector = tfidf_vec.reshape((dimension,))
        engine.store_vector(vector, index)

# добавим вектора
store_vectors(data_df, engine)

In [27]:
# функция для считывания данных (в данном случае названий статей) по выводу engine
def get_samples(df, neighbors, feature='title'):
    samples = []
    for neighbor in neighbors:
        vec, index, res = neighbor
        value = df.iloc[index][feature]
        samples.append(value)
    return samples

In [28]:
# Функция для запрашивания ближайших соседей данного названия (с необходимой предобработкой)
def query(engine, sample, samples_only=True):
    tfidf_vec = fasttext_tfidf([[sample]])[0]
    vector = tfidf_vec.reshape((dimension,))
    neighbors = engine.neighbours(vector)
    if samples_only:
        samples = get_samples(data_df, neighbors)
        return samples
    return neighbors

In [41]:
# достанем небольшую выборку, чтобы потестить на ней модель
rand_samples = data_df.sample(n=5, random_state=19)

In [37]:
# выведим результаты для выборки
for index, row in rand_samples.iterrows():
    title = row['title']
    neighbors = query(engine, title)
    print('Запрос статьи:', title)
    for sample in neighbors:
        print('Похожая статья:', sample)
    print('='*50)

Запрос статьи: В Сочи автомобиль протаранил остановку с людьми, двое погибли
Похожая статья: В Сочи автомобиль протаранил остановку с людьми, двое погибли
Похожая статья: В США пьяная водитель внедорожника протаранила автобус "Свидетелей Иеговы"
Похожая статья: Грузовик протаранил рейсовый автобус с пассажирами: один человек погиб, четверо ранены
Похожая статья: Могилев: маршрутка столкнулась с легковым автомобилем, тот перевернулся
Похожая статья: В центре Омска водитель джипа потерял сознание и протаранил машины
Похожая статья: В Омске пассажиры проломили голову водителю и угнали его автомобиль
Похожая статья: На юге Москвы автомобиль въехал в автобусную остановку
Похожая статья: Два человека погибли при столкновении автомобиля с остановкой в Москве
Похожая статья: Автомобиль сбил четырёх человек на остановке в Москве, двое погибли
Похожая статья: В Ростове автомойщик перепутал педали и протаранил дом иномаркой клиента
Запрос статьи: Сатирические рисунки, которые заставят вас улыбнут

Не обращайте внимание на то, что первые значения те же самые, что сами запросы. Просто они уже присутствуют в системе. Давайте попробуем дать запросы несуществующих в engine статей.

In [60]:
results = query(engine, 'В среду в Петербурге сохранится сухая погода')
for res in results:
    print(res)

С середины недели в Петербурге пойдет мокрый снег
Министр Силуанов пообещал, что цены на водку сильно не изменятся - Бизнес - Новости Санкт-Петербурга - Фонтанка.Ру
На территории Кировского завода в Петербурге произошел пожар
В Санкт-Петербурге школьница впала в кому на стадионе
Синоптики: штормовой ветер сохранится
На востоке Петербурга дотла сгорело офисное здание
В Петербурге встретятся лидеры газовой отрасли
Сильный туман в Петербурге увеличил число ДТП, но не повлиял на «Пулково»
Юг Петербурга затопило кипятком из лопнувшей трубы
Пассажиры задержанного «Сапсана» прибудут в Петербург с опозданием в три часа


In [61]:
results = query(engine, 'Политика: У США нет оснований изображать себя победителем в войне с террором')
for res in results:
    print(res)

Режим зеркала. Мы посмотрели, как поживает первая в стране интерактивная остановка спустя месяц после открытия (15 фото)
Граффити с изображением лидера донецких сепаратистов Арсения Павлова по прозвищу Моторола появилось в Петербурге
Комментарии - Россия наложила вето на проект резолюции об установлении бесполетной зоны над Алеппо - Delfi
Украина стала ассоциированным членом Европейской организации ядерных исследований
Эксперт: Борьба с кибератаками возможна при совместных усилиях РФ и США
В Пентагоне заявили, что примут меры в связи с размещением С-300 в Сирии
C новым стандартом IEEE скорость интернета в обычных кабелельных сетях вырастет в 5 раз
Открытие студии мобильной разработки «с нуля» в Питере — 3.5 года спустя. Реинкарнация. Часть 2 / Блог компании tapki.com
Норвежские лыжники пользуются небулайзерами в вакс-трейлере для приема медикаментов во время соревнований
«Южмаш» пообещал отгрузить ракету-носитель «Зенит» для «Морского старта» в 2017 году


In [40]:
results = query(engine, 'Фаната из Тулы будут судить за файер на матче с «Зенитом»')
for res in results:
    print(res)

Фаната из Тулы будут судить за файер на матче с «Зенитом»
Мендес: игра с «Зенитом» стала хорошей проверкой для «Сада Крузейро»
Премию Гавела присудили экс-пленнице "ИД", над которой жестоко издевались боевики
Федецкий признался, что ему тяжело смотреть за матчем сборной Украины со стороны
Тренер «Сада Крузейро» — о победе над «Зенитом»: это был идеальный матч
СМИ: Абрамович хочет купить защитника «Ювентуса» за рекордную сумму
Источник: матч «Зенит» — «Спартак» изначально должен был судить другой арбитр
«Он — человек ближайших окрестностей Земли»: Астроном Владимир Сурдин вспоминает своего коллегу Клима Чурюмова, который открыл самую знаменитую комету — Meduza
Гимаев: было бы замечательно увидеть в «Сент-Луисе» связку Тарасенко — Якупов
Трамп и Клинтон не пожали друг другу руки перед началом финальных дебатов


Как мы можем видеть по результам. Предсказания часто очень похожи на запрашиваемые статьи. Однако, есть статьи которые не очень подходят по смыслу и все же нужно признать, что алгоритм может быть улучшен. Ну, всегда есть к чему стремится!

Коротко о том, что хотелось бы улучшить в данном решении и вообще в решении задачи в целом:

- взять весь датасет, а не только часть;
- вместо тайтлов брать сами тексты, так как они содержать больше информации о том, о чем статья;
- использовать ELMo эмбеддинг, так как он context-sensitive и по сути является state-of-the-art решением (не считая Bert) для эмбеддинга;
- использовать иную модель для кластеризации. Возможно применение convolutional NN;
- хотелось бы оценить работу по каким-либо метрикам. Однако, это может оказаться задачей не из легких в данной задаче. Но хотя бы можно было бы взять скоростную метрику как один из вариантов.
- вообще было бы хорошо использовать и иные признаки, такие как время публикации, популярность в соцсетях и сам сайт. Из личного опыта, могу сказать, что последняя идея (на счет сайта) мне не нравится, ибо, к примеру, Google Chrome предлагает мне постоянно статьи с сайтов, которые я когда-то посещал, но больше не интересуюсь ими и это слегка раздражает меня, как пользователя.

Чтож, здесь на этом все. Теперь мы можем перейти к небольшой имплементации системы рекомендаций сайтов, которая реализована в папке ./system, также есть небольшой скрипт для демонстрации - файл test_system.py.

Хочу также отметить, что задача состояла в том, что у пользователя есть история предыдущих статей. Здесь, мы показали пример, когда у пользователя 1 статья. Чтобы масштабировать, я решил что буду предлагать пользователю до 10 статей, каждая из которых равновероятно может подходить одной из статей, прочитанных пользователем. Также, я ограничу историю статей пользователя до 10 статей, ибо не хотелось бы учитывать слишком старые статьи для рекомендаций. Все это реализовано в ранее указанных директории и файле.

### References
- PyMystem3: https://github.com/nlpub/pymystem3
- Препроцессинг русского текста: https://www.kaggle.com/alxmamaev/how-to-easy-preprocess-russian-text
- NearPy фреймворк: https://github.com/pixelogik/NearPy
- Визуализация данных с Plotly: https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
- Эмбеддеры из DeepPavlov: http://docs.deeppavlov.ai/en/latest/apiref/models/embedders.html
- Предобученные модели эмбеддинга для русского языка: http://docs.deeppavlov.ai/en/latest/intro/pretrained_vectors.html
